In [10]:
import time
import queue
from pysnmp.hlapi.asyncore import *
t = time.time()
myq = queue.Queue()

def getOID(strword):
    s1 = "."
    strword = strword.replace("SNMPv2-SMI::mib-2", "1.3.6.1.2.1")
    arr = strword.split(".")
    del arr[-1]
    return s1.join(arr)

#回调函数。在有数据返回时触发
def cbFun(snmpEngine, sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
    myq.put((time.time()-t, varBinds, errorIndication, errorStatus, errorIndex))
hosts = ["140.134.207.74","140.134.207.73"]

oids = ['1.3.6.1.2.1.1','1.3.6.1.2.1.2.2','1.3.6.1.2.1.25.1','1.3.6.1.2.1.25.2','1.3.6.1.2.1.25.4','1.3.6.1.2.1.25.4.2.1.1','1.3.6.1.2.1.25.4.2.1.2','1.3.6.1.2.1.25.5','1.3.6.1.2.1.25.6']
        
snmpEngine = SnmpEngine()

#添加任务
for oid in oids:
    for h in hosts:
        print("ip:", h)
        nextCmd(snmpEngine,
            CommunityData('rmc2772'),
            UdpTransportTarget((h, 161), timeout=3, retries=0,),
            ContextData(),
            ObjectType(ObjectIdentity(oid)),
            cbFun=cbFun)
time1 = time.time() - t

#执行异步获取snmp
snmpEngine.transportDispatcher.runDispatcher()

#打印结果
while True:
    try:
        info = myq.get(block=False)
#         print(info)
        if info[2]:  # SNMP agent errors
            print("errorIndication:", info[2])
        else:
            if info[3]:
                print("here")
                print('%s at %s' % (info[3].prettyPrint(), info[1][int(info[4])-1] if info[4] else '?'))
            else:
                for row in info[1]:
                    #print("row:", row)
                    for oid, val in row:
                        print("oid:", getOID(oid.prettyPrint()), "val:", val.prettyPrint())
    except queue.Empty:
        print(time1)
        print(time.time() - t)
        break

ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
ip: 140.134.207.74
ip: 140.134.207.73
oid: SNMPv2-MIB::sysDescr val: Hardware: Intel64 Family 6 Model 158 Stepping 10 AT/AT COMPATIBLE - Software: Windows Version 6.3 (Build 18362 Multiprocessor Free)
oid: SNMPv2-MIB::sysDescr val: Hardware: Intel64 Family 6 Model 158 Stepping 10 AT/AT COMPATIBLE - Software: Windows Version 6.3 (Build 18362 Multiprocessor Free)
oid: 1.3.6.1.2.1.2.2.1.1 val: 1
oid: 1.3.6.1.2.1.25.1.1 val: 1145292
oid: 1.3.6.1.2.1.25.2.2 val: 16568528
oid: 1.3.6.1.2.1.2.2.1.1 val: 1
oid: 1.3.6.1.2.1.25.1.1 val: 1145273
oid: 1.3.6.1.2.1.25.2.2 val: 16568528
oid: 1.3.6.1.2.1.25.4.1 val: 4
oid: 1.3.6.1.2.1.25.4.2.1.1 val: 1
oid: 1.3.6.1.2.1.25.4.2.1.2 val: System Idle Process